In [1]:
import numpy as np
import pandas as pd
from linearmodels.iv import IV2SLS
from statsmodels.api import add_constant

In [2]:
np.seterr(divide = 'ignore')

df = pd.read_csv('source/csad-stocks.csv')
df = df.dropna()

# add constant
df['index'] = df.index 
df = add_constant(df, has_constant='add')

df.head()

,const,Datadate,MktRf,Smb,Hml,Rmw,Cma,Rf,Mom,Rm,AbsoluteRm,SquaredRm,Csad,index
1,1.0,2002-02-28,-0.001153,-0.000232,0.001205,0.004358,0.002695,0.00007,0.002826,-0.001083,0.001083,1.172091e-06,0.082511,1
2,1.0,2002-03-31,0.002135,0.001985,0.000500,-0.000895,0.000255,0.00007,-0.000380,0.002205,0.002205,4.862025e-06,0.082475,2
3,1.0,2002-04-30,-0.002382,0.003118,0.001764,0.002077,0.002418,0.00007,0.003732,-0.002312,0.002312,5.344503e-06,0.013145,3
4,1.0,2002-05-31,-0.000545,-0.001436,0.000655,0.001023,0.001077,0.00007,0.001668,-0.000475,0.000475,2.260570e-07,0.012443,4
5,1.0,2002-06-30,-0.003675,0.002010,0.000010,0.002040,0.001275,0.00006,0.003695,-0.003615,0.003615,1.306822e-05,0.052099,5


## Model 1

In [3]:
res_ols_v1 = IV2SLS(
    df.Csad, df[
        ['Rm', 'AbsoluteRm', 'SquaredRm', 'index', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v1)

                            OLS Estimation Summary                            
Dep. Variable:                   Csad   R-squared:                      0.5300
Estimator:                        OLS   Adj. R-squared:                 0.5222
No. Observations:                 245   F-statistic:                    276.27
Date:                Sat, Sep 16 2023   P-value (F-stat)                0.0000
Time:                        17:12:32   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Rm             4.9593     0.9483     5.2298     0.0000      3.1007      6.8179
AbsoluteRm     8.9916     4.2394     2.1209     0.03

## Model 2

In [4]:
res_ols_v21 = IV2SLS(
    df.Csad, df[
        ['MktRf', 'Smb', 'Hml', 'Rmw', 'Cma', 'index', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v21)

                            OLS Estimation Summary                            
Dep. Variable:                   Csad   R-squared:                      0.1223
Estimator:                        OLS   Adj. R-squared:                 0.1002
No. Observations:                 245   F-statistic:                    34.132
Date:                Sat, Sep 16 2023   P-value (F-stat)                0.0000
Time:                        17:12:32   Distribution:                  chi2(6)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
MktRf          4.2559     1.4445     2.9464     0.0032      1.4248      7.0871
Smb            5.9771     2.4606     2.4291     0.01

In [5]:
res_ols_v22 = IV2SLS(
    df.Csad, df[
        ['MktRf', 'Smb', 'Hml', 'Mom', 'index', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v22)

                            OLS Estimation Summary                            
Dep. Variable:                   Csad   R-squared:                      0.1258
Estimator:                        OLS   Adj. R-squared:                 0.1075
No. Observations:                 245   F-statistic:                    35.249
Date:                Sat, Sep 16 2023   P-value (F-stat)                0.0000
Time:                        17:12:32   Distribution:                  chi2(5)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
MktRf          0.1508     1.5171     0.0994     0.9208     -2.8226      3.1242
Smb            5.3691     2.3710     2.2645     0.02

## Model 3

In [6]:
df['Csad_Fund'] = df['Csad'] - res_ols_v21.resids

res_ols_v3 = IV2SLS(
    df.Csad_Fund, df[
        ['Rm', 'AbsoluteRm', 'SquaredRm', 'index', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v3)

                            OLS Estimation Summary                            
Dep. Variable:              Csad_Fund   R-squared:                      0.4141
Estimator:                        OLS   Adj. R-squared:                 0.4043
No. Observations:                 245   F-statistic:                    173.14
Date:                Sat, Sep 16 2023   P-value (F-stat)                0.0000
Time:                        17:12:32   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Rm             3.9468     0.3702     10.660     0.0000      3.2212      4.6725
AbsoluteRm    -1.0792     1.6552    -0.6520     0.51

In [7]:
df['Csad_Fund'] = df['Csad'] - res_ols_v22.resids

res_ols_v3 = IV2SLS(
    df.Csad_Fund, df[
        ['Rm', 'AbsoluteRm', 'SquaredRm', 'index', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v3)

                            OLS Estimation Summary                            
Dep. Variable:              Csad_Fund   R-squared:                      0.4288
Estimator:                        OLS   Adj. R-squared:                 0.4192
No. Observations:                 245   F-statistic:                    183.89
Date:                Sat, Sep 16 2023   P-value (F-stat)                0.0000
Time:                        17:12:32   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Rm             4.0123     0.3708     10.822     0.0000      3.2856      4.7389
AbsoluteRm    -0.9533     1.6575    -0.5751     0.56

## Model 4

In [8]:
df['CSAD_nonFund'] = res_ols_v21.resids

res_ols_v4 = IV2SLS(
    df.CSAD_nonFund, df[
        ['Rm', 'AbsoluteRm', 'SquaredRm', 'index', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v4)

                            OLS Estimation Summary                            
Dep. Variable:           CSAD_nonFund   R-squared:                      0.4965
Estimator:                        OLS   Adj. R-squared:                 0.4881
No. Observations:                 245   F-statistic:                    241.55
Date:                Sat, Sep 16 2023   P-value (F-stat)                0.0000
Time:                        17:12:32   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Rm             1.0125     0.9196     1.1010     0.2709     -0.7899      2.8148
AbsoluteRm     10.071     4.1111     2.4497     0.01

In [10]:
df['CSAD_nonFund'] = res_ols_v22.resids

res_ols_v4 = IV2SLS(
    df.CSAD_nonFund, df[
        ['Rm', 'AbsoluteRm', 'SquaredRm', 'index', 'const']
    ], None, None
).fit(cov_type='unadjusted')

print(res_ols_v4)

                            OLS Estimation Summary                            
Dep. Variable:           CSAD_nonFund   R-squared:                      0.4519
Estimator:                        OLS   Adj. R-squared:                 0.4428
No. Observations:                 245   F-statistic:                    202.01
Date:                Sat, Sep 16 2023   P-value (F-stat)                0.0000
Time:                        17:13:13   Distribution:                  chi2(4)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Rm             0.9471     0.9575     0.9891     0.3226     -0.9295      2.8237
AbsoluteRm     9.9449     4.2805     2.3233     0.02